<a href="https://colab.research.google.com/github/Thaleslsilva/DataScience/blob/master/CBOW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Estudo de Caso - Inteligência Artificial Para Previsão de Sentenças em Embargos de Declaração

In [ ]:
# Para atualizar um pacote, execute o comando abaixo no terminal ou prompt de comando:
# pip install -U nome_pacote

# Para instalar a versão exata de um pacote, execute o comando abaixo no terminal ou prompt de comando:
# !pip install torch==1.5.0

# Depois de instalar ou atualizar o pacote, reinicie o jupyter notebook.

# Instala o pacote watermark. 
# Esse pacote é usado para gravar as versões de outros pacotes usados neste jupyter notebook.
!pip install -q -U watermark

In [ ]:
# Instala o PyTorch
!pip install -q torch 

In [ ]:
# Imports
import torch
import torch.nn as nn
import numpy as np

In [ ]:
# Versões dos pacotes usados neste jupyter notebook
%reload_ext watermark
%watermark -a "Thales de Lima Silva" --iversions

### Preparando os Dados

O texto abaixo é um exemplo de embargo de declaração. Embora o texto represente um embargo, dados críticos foram substituídos por informações genéricas, o que não compromete o objetivo do estudo de caso.

In [ ]:
# Texto de embargo de declaração
embargo = """O embargante sofreu o ajuizamento de ação de danos morais e materiais, cujo objeto é o reaver os 
valores pagos pelo sinal dado em um contrato de compra e venda de imóvel no qual não foi dado continuidade. Em 24 
de fevereiro de 2012, o Magistrado proferiu decisão de fls. 277 a 280, que condenou todas as demandadas 
solidariamente no seguinte teor: Diante de todo o exposto, com fundamento no art. 1234, I, do CPC/2015, 
julgo procedentes em parte os pedidos constantes na inicial, condenando solidariamente as demandadas, XPTO LTDA, 
BOB CAMARGO DE MORAES, a Pagarema título de indenização por danos morais, consoante fundamentação acima discorrida, 
o montante de R$ 1.500,00 (um mil e quinhentos reais), corrigidos monetariamente pelo INPC desde a data 
desta decisão, acrescidos de juros de 1% ao mês, a partir da citação; condeno ainda, à restituição do valor 
pago pelo demandante como sinal da entrada do imóvel, descontando apenas 20% (vinte por cento), referente às 
despesas, devendo incidir juros de 1% (um por cento) ao mês contados da citação e correção monetária pelo INPC a 
partir da sentença. Contudo, data venia, houve omissão e obscuridade na referida decisão, haja vista que a omissão 
se deu pela ausência dos julgamentos das preliminares (Necessidade de Perícia Técnica e a incompetência de 
Juizado Especial) proposta posteriormente em aditamento de contestação (Fls 251 a 254) para impugnar áudios 
juntados pelo embargado, autorizado a ser realizada pela Douta Magistrada em audiência de Conciliação, 
instrução e julgamento de fls 235 e 236, por ausência de intimação anterior para realizar a já tratada 
impugnação aos áudios anexados."""

In [ ]:
# Limpeza do texto substituindo vírgulas e pontos por espaços e colocando as palavras em minúsculo
embargo = embargo.replace(',','').replace('.','').lower().split()

In [ ]:
# Criação do corpus com o texto acima
corpus = set(embargo)

In [ ]:
# Visualizamos o corpus
corpus

In [ ]:
# Comprimento do corpus
corpus_length = len(corpus)

In [ ]:
# Dicionários para TF-IDF
dic_palavra = {}
dic_inverso_palavra = {}

In [ ]:
# Loop pelo corpus para criar os dicionários
for i, palavra in enumerate(corpus):
    dic_palavra[palavra] = i
    dic_inverso_palavra[i] = palavra

In [ ]:
# Lista para receber os dados
dados = []

In [ ]:
# Loop pelo texto par extrair sentenças e palavras
for i in range(2, len(embargo) - 2):
    sentence = [embargo[i-2], embargo[i-1], embargo[i+1], embargo[i+2]]
    target = embargo[i]
    dados.append((sentence, target))

Você leu o código acima e compreendeu o que foi feito? Observe esta linha:

sentence = [embargo[i-2], embargo[i-1], embargo[i+1], embargo[i+2]]

Para uma palavra no índice i, obtemos duas palavras antes e duas palavras depois. A palavra no índice i será o nosso target e a sentença será composta das duas palavras e duas palavras depois da palavra target. 

Após treinar o modelo, seremos capazes de prever cada palavra com base nas palavras a sua volta.

Aqui um exemplo:

In [ ]:
# Visualiza os dados
print(dados[3])

As quatro palavras na lista serão os dados de entrada e a palavra fora da lista ('de' nesse caso), será a variável de saída.

### Construção do Modelo CBoW

In [ ]:
# Vamos definir o comprimento de cada embedding
embedding_length = 20

In [ ]:
# Classe para o modelo
class CBoW(torch.nn.Module):

    # Método construtor
    def __init__(self, corpus_length, embedding_dim):
        super(CBoW, self).__init__()
        
        # Camada de entrada do modelo para criação da embedding
        self.embeddings = nn.Embedding(corpus_length, embedding_dim)

        # Camadas lineares
        self.linear1 = nn.Linear(embedding_dim, 64)
        self.linear2 = nn.Linear(64, corpus_length)
        
        # Camadas de ativação
        self.activation_function1 = nn.ReLU()
        self.activation_function2 = nn.LogSoftmax(dim = -1)

    # Passo (forward)
    def forward(self, inputs):
        
        # Aqui definimos a ordem ds camadas da rede neural
        embeds = sum(self.embeddings(inputs)).view(1,-1)
        out = self.linear1(embeds)
        out = self.activation_function1(out)
        out = self.linear2(out)
        out = self.activation_function2(out)
        return out

    # Obtém a word_emdedding
    def get_word_emdedding(self, word):
        word = torch.LongTensor([dic_palavra[word]])
        return self.embeddings(word).view(1,-1)

In [ ]:
# Cria o modelo CBoW
modelo = CBoW(corpus_length, embedding_length)

In [ ]:
# Função de custo
loss_function = nn.NLLLoss()

In [ ]:
# Otimizador do modelo (backpropagation)
optimizer = torch.optim.SGD(modelo.parameters(), lr = 0.01)

In [ ]:
# Função para criar o vetor de sentenças, necessário para treinar o modelo
def make_sentence_vector(sentence, word_dict):
    idxs = [word_dict[w] for w in sentence]
    return torch.tensor(idxs, dtype = torch.long)

In [ ]:
# Aqui está nosso dicionário de palavras
dic_palavra

In [ ]:
# O dicionário de palavras será convertido em um vetor de sentenças. Aqui um exemplo:
print(make_sentence_vector(['pela','ausência','dos','julgamentos'], dic_palavra))

### Treinamento do Modelo

In [ ]:
# Loop por 150 passadas (epochs) de treinamento
for epoch in range(150):
    
    # Inicia o erro da época com 0
    epoch_loss = 0
    
    # Loop pelos dados de entrada (sentence) e saída (target)
    for sentence, target in dados:
        
        # Inicializa os gradientes com zero
        modelo.zero_grad()
        
        # Cria o vetor de sentença com os dados de entrada (que devem estar no dicionário de palavras)
        sentence_vector = make_sentence_vector(sentence, dic_palavra)  
        
        # Usa o vetor para fazer previsões com o modelo e retorna as probabilidades
        log_probs = modelo(sentence_vector)
        
        # Calcula o erro do modelo
        loss = loss_function(log_probs, torch.tensor([dic_palavra[target]], dtype = torch.long))
        
        # Chama o método de backpropagation para calcular o gradiente da derivada
        loss.backward()
        
        # Otimiza os pesos do modelo e segue para a próxima passada
        # É aqui que o aprendizado acontece
        optimizer.step()
        
        # Atualiza o erro da época
        epoch_loss += loss.data
        
    # Imprime epoch e erro da epoch    
    print('Epoch: ' + str(epoch) + ', Erro do Modelo: ' + str(epoch_loss.item()))

Observe como o erro foi reduzido a cada passada, nitidamente o aprendizado ocorrendo. Vamos agora usar o modelo para fazer previsões.

In [ ]:
# Função para obter uma previsão
def get_resultado_previsto(input, dic_inverso_palavra):
    index = np.argmax(input)
    return dic_inverso_palavra[index]

In [ ]:
# Função para prever sentenças (aplicamos aos novos dados o mesmo tratamento usado nos dados de treino)
def preve_sentenca(sentence):
    
    # Dividimos a sentença com split
    sentence_split = sentence.replace('.','').lower().split()
    
    # Criamos o vetor de sentença
    sentence_vector = make_sentence_vector(sentence_split, dic_palavra)
    
    # Faz a previsão com o modelo
    prediction_array = modelo(sentence_vector).data.numpy()
    
    # Print dos resultados
    print('Palavras Anteriores: {}\n'.format(sentence_split[:2]))
    print('Palavra Prevista: {}\n'.format(get_resultado_previsto(prediction_array[0], dic_inverso_palavra)))
    print('Palavras Seguintes: {}\n'.format(sentence_split[2:]))

### Previsões com o Modelo

Dentro da frase: **"ausência de intimação anterior para realizar"**, vejamos se o modelo consegue prever a palavra.

Vou omitir a palavra **intimação** e essa deve ser a palavra prevista pelo modelo. Vamos passar como dados de entrada as duas palavras anteriores e as duas palavras posteriores.

In [ ]:
# Previsão com o modelo
preve_sentenca('ausência de anterior para')

In [ ]:
# Emdedding da palavra
print(modelo.get_word_emdedding('intimação'))

Dentro da frase: **"devendo incidir juros de 1%"**, vejamos se o modelo consegue prever a palavra.

Vou omitir a palavra **juros** e essa deve ser a palavra prevista pelo modelo. Vamos passar como dados de entrada as duas palavras anteriores e as duas palavras posteriores.

In [ ]:
# Previsão com o modelo
preve_sentenca('devendo incidir de 1%')

# Fim